# Limpeza — demographics.csv 
esse caderno, por enquanto, apenas organiza em ordem crescente as informações, e retira duplicatas

In [ ]:
from pathlib import Path
# (ajuste para o seu usuário/sistema):
CSV_PATH = Path("/Users/augusto/Library/Mobile Documents/com~apple~CloudDocs/git/avalensurance-bia/data/demographics.csv")


print("CSV_PATH:", CSV_PATH)
print("Existe o arquivo?", CSV_PATH.exists())
if not CSV_PATH.exists():
    raise FileNotFoundError("Arquivo não encontrado. Edite CSV_PATH acima e rode novamente.")

In [ ]:
import pandas as pd

df = pd.read_csv(CSV_PATH)
print("Dimensão inicial (linhas, colunas):", df.shape)

if "person_id" not in df.columns:
    raise ValueError("A coluna 'person_id' não existe no CSV.")

# Ordenar por person_id (mergesort estável): numérico -> string
pid_num = pd.to_numeric(df["person_id"], errors="coerce")
df = (
    df.assign(_pid_num=pid_num, _pid_str=df["person_id"].astype(str))
      .sort_values(by=["_pid_num", "_pid_str"], kind="mergesort", na_position="last")
      .drop(columns=["_pid_num", "_pid_str"])
      .reset_index(drop=True)
)

# Remover duplicatas EXATAS
before = len(df)
df = df.drop_duplicates(keep="first")
after = len(df)
print(f"Duplicatas exatas removidas: {before - after} (de {before} → {after})")

# Contradições (mesmo person_id com outros campos diferentes)
other_cols = [c for c in df.columns if c != "person_id"]
contradiction_ids = []
if other_cols:
    unique_combos = df.drop_duplicates(subset=["person_id"] + other_cols)
    counts = unique_combos.groupby("person_id", dropna=False).size()
    contradiction_ids = counts[counts > 1].index.tolist()

# Caminhos de saída com base na raiz do repo (um nível acima de 'data')
repo_dir = CSV_PATH.parent.parent
out_clean = repo_dir / "data" / "demographics_cleaned.csv"
out_contra = repo_dir / "data" / "person_ids_contradicoes.csv"

# Salvar
out_clean.parent.mkdir(parents=True, exist_ok=True)
df.to_csv(out_clean, index=False)
print("✅ CSV limpo salvo em:", out_clean)

if contradiction_ids:
    pd.DataFrame({"person_id": contradiction_ids}).to_csv(out_contra, index=False)
    print(f"⚠️ {len(contradiction_ids)} person_id(s) com contradições. Lista salva em:", out_contra)
    for pid in contradiction_ids[:100]:
        print(f"person_id::{pid}  erro contraditório")
    if len(contradiction_ids) > 100:
        print(f"... e mais {len(contradiction_ids) - 100} person_id(s).")
else:
    print("✔ Nenhuma contradição encontrada.")